# 0. what for
1. 对大数据集计算np
# 1. Preparations
## 1.1 全局设置

In [218]:
#导入pandas包
import pandas as pd
import numpy as np

auctions_cleaned_path = r'E:\DATA\large_dta\auctions_cleaned.csv'   # retail修正的data
auctions_threshold = r'E:\DATA\large_dta\auctions_threshold.csv'    # 暂存做完threshold的data但是没用到

# settings path after threshold
settings_GT_path_thld = r'E:\DATA\large_dta\settings_GT_thld.csv'
settings_NN_path_thld = r'E:\DATA\large_dta\settings_NN_thld.csv'

# final data output
data_np_path = r'E:\DATA\large_dta\auctions_np.csv'                 #
settings_np_path = r'E:\DATA\large_dta\settings_np.csv'             #
prod_path = r'E:\DATA\large_dta\prod.csv'

- 读取

In [219]:
data = pd.read_csv(auctions_cleaned_path,encoding="utf-8")

unique_setting_GT = ['k','y','retail']
unique_setting_NN = ['item','k','y','retail']

In [220]:
threshold = 16

# 1. threshold
1. threshold筛选的是某个unique_setting_NN下，记录不够多，那么这个setting不具有代表性
2.

In [221]:
data.head()

,auctionid,pennyauction,fixedprice,type,currency,exchangerate,adjvalue,endtimeperiod,k,y,retail,item
0,138823,1,0,Consumer Goods,Euro,1.3978,9.546973,10.0,0.01,0.01,10.0,COSMETICS Brush SET 6 PIECES MADE OF WOOD
1,175928,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER
2,176047,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER
3,176073,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0,20 BIDS VOUCHER
4,176010,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0,20 BIDS VOUCHER


- group by操作，计算cnt_total: 这个settings下一共出现了多少场auctions

In [222]:
features = unique_setting_NN
data_NN = data.groupby(by = features,as_index=False)
# cnt_total = data_NN.endtimeperiod.sum()
# cnt_total
data_NN_grouped = pd.DataFrame(data_NN.size())
data_NN_grouped.rename(columns={'size': 'cnt_uniq'}, inplace=True)
print(data_NN_grouped.head())
data_NN_grouped.shape

                         item     k     y  retail  cnt_uniq
0             1 KG SILVER BAR  0.01  0.01   559.0        33
1       10 GRAMS OF GOLD BARS  0.01  0.01   325.0        21
2      10 GRAMS PLATINUM BARS  0.01  0.01   366.0         7
3  10 MARKS BAVARIA GOLD COIN  0.01  0.01   268.0         5
4      1000 FREEBIDS VOUCHER!  0.01  0.01   671.0         3


(740, 5)

- cnt_uniq加入大表，筛下去那些小于threshold的settings

In [223]:
data_cnt = pd.merge(data,data_NN_grouped,on=unique_setting_NN,how="left")
data_thrld = data_cnt[data_cnt.cnt_uniq>=threshold].copy()

settings_NN_thrld = data_thrld[unique_setting_NN].drop_duplicates(inplace=False)
settings_NN_thrld.reset_index(drop=True,inplace=True)
n_settings = settings_NN_thrld.shape[0]
print(f"当threshold = *{threshold}* 时，剩余{n_settings}个 auction settings")

当threshold = *16* 时，剩余78个 auction settings


In [224]:
settings_NN_thrld.to_csv(settings_np_path,encoding="utf-8",index=False,header=True)
settings_NN_thrld.head()

,item,k,y,retail
0,20 BIDS VOUCHER,0.01,0.01,14.0
1,20 BIDS VOUCHER,0.01,0.01,15.0
2,50 BIDS VOUCHER,0.01,0.01,35.0
3,50 FREEBIDS VOUCHER!,0.01,0.01,35.0
4,MAKITA CORDLESS ARTICULATED WRENCH,0.01,0.01,59.0


# 2. 计算np
## 2.1 计算 'n'
1. 'endtimeperiod'记录了这个auction的duration，上面计算了这个duration的出现频率

In [225]:
data_thrld.rename(columns={"endtimeperiod":"N"},inplace=True)
data_thrld.head()

,auctionid,pennyauction,fixedprice,type,currency,exchangerate,adjvalue,N,k,y,retail,item,cnt_uniq
1,175928,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0
2,176047,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0
3,176073,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0
4,176010,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0
5,175995,1,0,Consumer Goods,Euro,1.3616,13.616000,16.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0


## 2.2 计算'P'
### 2.2.1 计算'cnt_N'
1. 应该计算当前setting下，不同的‘N’值出现的次数'cnt_N'

In [226]:
features = unique_setting_NN+['N']
data_grouped = data_thrld.groupby(by=features,as_index=False)
data_cnt_N = pd.DataFrame(data_grouped.size())
data_cnt_N.rename(columns={'size': 'cnt_N'}, inplace=True)

data_P = pd.merge(data_thrld,data_cnt_N,on=features,how="left")
data_P.head()
# pd.merge(data_thrld,data_cnt_N,on=features,how="")

,auctionid,pennyauction,fixedprice,type,currency,exchangerate,adjvalue,N,k,y,retail,item,cnt_uniq,cnt_N
0,175928,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0,10
1,176047,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0,10
2,176073,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0,15
3,176010,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0,6
4,175995,1,0,Consumer Goods,Euro,1.3616,13.616000,16.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0,6


### 2.2.2 计算P

In [227]:
data_P['P'] = data_P['cnt_N'] / data_P['cnt_uniq']
data_P.head()

,auctionid,pennyauction,fixedprice,type,currency,exchangerate,adjvalue,N,k,y,retail,item,cnt_uniq,cnt_N,P
0,175928,1,0,Consumer Goods,Euro,1.3500,13.500000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0,10,0.026525
1,176047,1,0,Consumer Goods,Euro,1.3448,13.448000,18.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0,10,0.026525
2,176073,1,0,Consumer Goods,Euro,1.3305,13.305000,15.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0,15,0.039788
3,176010,1,0,Consumer Goods,Euro,1.3640,13.639999,32.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0,6,0.015915
4,175995,1,0,Consumer Goods,Euro,1.3616,13.616000,16.0,0.01,0.01,14.0,20 BIDS VOUCHER,377.0,6,0.015915


In [228]:
features = unique_setting_NN +['N'] + ['P']+['cnt_N']
data_P = data_P[features].copy()
data_P.head()

,item,k,y,retail,N,P,cnt_N
0,20 BIDS VOUCHER,0.01,0.01,14.0,18.0,0.026525,10
1,20 BIDS VOUCHER,0.01,0.01,14.0,18.0,0.026525,10
2,20 BIDS VOUCHER,0.01,0.01,14.0,15.0,0.039788,15
3,20 BIDS VOUCHER,0.01,0.01,14.0,32.0,0.015915,6
4,20 BIDS VOUCHER,0.01,0.01,14.0,16.0,0.015915,6


- 保存data
- 保存item方便embedding

In [229]:
data_P.to_csv(data_np_path,encoding="utf-8",index=False,header=True)

In [230]:
item_df = pd.DataFrame(data_P['item'].drop_duplicates(inplace=False))
item_df.reset_index(drop=False,inplace=True)
item_df.to_csv(prod_path,encoding="utf-8",index=False,header=True)